# Demo notebook for "Finding Data for your Research Organism: Plants and RNASeq Data"

This notebook shows the commands to replicate the demo provided during the NCBI Webinar on August 18,2021

## Workflow

### 1.) Fastq files are generated from an accession list using NCBI SRA Toolkit
### 2.) A reference genome is downloaded using NCBI Datasets command line 
### 3.) The sequences in the fastq files are aligned using NCBI's Magic-BLAST tool
### 4.) Once files are generated, they can be loaded into the NCBI Genome Data Viewer

A few notes:

* This demo is intended to be run on binder as all programmatic installs have been done directly to the binder server
* If you wish to download and use this notebook on your own server, please find the instructions for the programmatic installs in the README.md file for this git repo: https://github.com/Arhodes-NCBI/conda/blob/master/README.md
* The commands in this notebook that begin with "!" are bash commands that can be copied and run directly in a terminal.

Command Line programs already loaded onto the server, but if you are running this notebook locally, you may need to install and add these programs to your path to work with these commands as written.

**SRA Toolkit** (install instructions here: https://github.com/ncbi/sra-tools/wiki/02.-Installing-SRA-Toolkit)

**Datasets Command Line** (install instructions here: https://www.ncbi.nlm.nih.gov/datasets/docs/quickstarts/command-line-tools/

**Blast+ Executables** (install instructions here: https://www.ncbi.nlm.nih.gov/books/NBK279690/)

**MagicBlast** (install instructions here: https://ncbi.github.io/magicblast/)

## Configure SRA Toolkit

If this is the first time running the notebook, use the following cell to set up SRA toolkit

In [18]:
GUID = !uuidgen
GUID = str(GUID).strip('[]')
!vdb-config --set LIBS/GUID={GUID}
!vdb-config --report-cloud-identity yes
!vdb-config --set /repository/user/main/public/cache-disabled=true
#!cat $HOME/.ncbi/user-settings.mkfg

Report Cloud Instance Identity was set to true


Test that the sra toolkit is configured and working from this location by downloading the first two reads with quality scores using an SRA Accession.

In [3]:
!fastq-dump --stdout -X 2 SRR3241527

Read 2 spots for SRR3241527
Written 2 spots for SRR3241527
@SRR3241527.1 FCC3E8RACXX:1:1101:8441:2000# length=100
NCCACCCTTCGAAATCACAGTCCGTGAGTCTCCAACATTGGCAACATAAAGATGATCACCAACTAGCACTGCTGTTGAAGCAGTAGAACCATCATCACGG
+SRR3241527.1 FCC3E8RACXX:1:1101:8441:2000# length=100
#4BDFFFFHHHHHJJJIJJJHIIJGHIIEGHIJJJJJJJJIJJJJJJJJJIJJJJJJJJJJJJJJJHHHHHFFFFDFEEEEEDCCDDDDDDCDDDDDDDB
@SRR3241527.2 FCC3E8RACXX:1:1101:9310:1999# length=100
NGTGAGCATCCGCAGGGACCATCGCAATGCTTTGTTTTAATTAAACAGTCGGATTCCCCTTGTCCGTACCAGTTCTGAGTCGACCGTTCGACGCCCGGGG
+SRR3241527.2 FCC3E8RACXX:1:1101:9310:1999# length=100
#4=DDFFFHHHHHJIHJJJIJIJJJJJIIJJJJIIIJJJIJJJJJJJJFIIJJJJHHHHEFFFFFDDEDDDDCDDCDDDDDDDDDBDDDDDDDDDD@9<B


## Load sequence data using SRA Toolkit command fastq-dump

For convenience, the accession list that we generated using the SRA Run Selector has been added to this notebook environment for you. You can reuse this notebook to run your own accessions (memory permitting) by just replacing this text file in the directory with your own list.

These 12 accessions belong to a Bioproject that measured RNA expression in Populus trichocarpus xylem on days 0, 5 and 7 of a drought treatment. More details can be found here: 

In [17]:
!cat accessionlist.txt

SRR3241527
SRR3241528
SRR3241529
SRR3241530
SRR3241531
SRR3241532
SRR3241533
SRR3241534
SRR3241535
SRR3241536
SRR3241537
SRR3241538


In [5]:
!cat accessionlist.txt | xargs -P 12 -n 1 fastq-dump -X 1000000 -O data

Read 1000000 spots for SRR3241531
Written 1000000 spots for SRR3241531
Read 1000000 spots for SRR3241538
Written 1000000 spots for SRR3241538
Read 1000000 spots for SRR3241533
Written 1000000 spots for SRR3241533
Read 1000000 spots for SRR3241532
Written 1000000 spots for SRR3241532
Read 1000000 spots for SRR3241527
Written 1000000 spots for SRR3241527
Read 1000000 spots for SRR3241534
Written 1000000 spots for SRR3241534
Read 1000000 spots for SRR3241530
Written 1000000 spots for SRR3241530
Read 1000000 spots for SRR3241529
Written 1000000 spots for SRR3241529
Read 1000000 spots for SRR3241535
Written 1000000 spots for SRR3241535
Read 1000000 spots for SRR3241536
Written 1000000 spots for SRR3241536
Read 1000000 spots for SRR3241537
Written 1000000 spots for SRR3241537
Read 1000000 spots for SRR3241528
Written 1000000 spots for SRR3241528


In [2]:
!ls data

SRR3241527.fastq  SRR3241530.fastq  SRR3241533.fastq  SRR3241536.fastq
SRR3241528.fastq  SRR3241531.fastq  SRR3241534.fastq  SRR3241537.fastq
SRR3241529.fastq  SRR3241532.fastq  SRR3241535.fastq  SRR3241538.fastq


## Download the RefSeq Genome and annotations from Datasets

The NCBI Datasets datasets command line tools are **datasets** and **dataformat**.

Use datasets to download biological sequence data across all domains of life from NCBI.

Use dataformat to convert metadata from JSON Lines format to other formats.

Note: The NCBI Datasets command line tools are currently in alpha and will be updated frequently to add new features, fix bugs, and enhance usability. Command syntax is subject to frequent changes. Please check back often for updates.

This part of the analysis repliates what we did with the web browser, except by using the command line tool provided by NCBI.

In [8]:
!datasets download genome accession GCF_000002775.4 --chromosomes 5 --include-gtf --exclude-protein --exclude-rna
 

Downloading: ncbi_dataset.zip    9.76kB 15.6MB/s
Downloading: ncbi_dataset.zip    9.76kB 15.6MB/s
Downloading: ncbi_dataset.zip    34.8kB 1.26MB/s
Downloading: ncbi_dataset.zip    34.8kB 1.26MB/s
Downloading: ncbi_dataset.zip    34.8kB 1.26MB/s
Downloading: ncbi_dataset.zip    87.7kB 1.6MB/s
Downloading: ncbi_dataset.zip    87.7kB 1.6MB/s
Downloading: ncbi_dataset.zip    187kB 2.29MB/s
Downloading: ncbi_dataset.zip    187kB 2.29MB/s
Downloading: ncbi_dataset.zip    187kB 2.29MB/s
Downloading: ncbi_dataset.zip    381kB 3.49MB/s
Downloading: ncbi_dataset.zip    381kB 3.49MB/s
Downloading: ncbi_dataset.zip    497kB 3.68MB/s
Downloading: ncbi_dataset.zip    781kB 5.68MB/s
Downloading: ncbi_dataset.zip    781kB 5.68MB/s
Downloading: ncbi_dataset.zip    1.27MB 7.74MB/s
Downloading: ncbi_dataset.zip    1.27MB 7.74MB/s
Downloading: ncbi_dataset.zip    1.27MB 7.74MB/s
Downloading: ncbi_dataset.zip    1.32MB 6.69MB/s
Downloading: ncbi_dataset.zip    2.55MB 12.5MB/s
Downloading: ncbi_dataset.zip 

Downloading: ncbi_dataset.zip    26.9MB 17.5MB/s
Downloading: ncbi_dataset.zip    27.1MB 17.5MB/s
Downloading: ncbi_dataset.zip    27.3MB 17.5MB/s
Downloading: ncbi_dataset.zip    27.4MB 17.5MB/s
Downloading: ncbi_dataset.zip    27.6MB 17.5MB/s
Downloading: ncbi_dataset.zip    27.7MB 17.4MB/s
Downloading: ncbi_dataset.zip    28MB 17.5MB/s
Downloading: ncbi_dataset.zip    28.1MB 17.5MB/s
Downloading: ncbi_dataset.zip    28.3MB 17.5MB/s
Downloading: ncbi_dataset.zip    28.5MB 17.5MB/s
Downloading: ncbi_dataset.zip    28.8MB 17.5MB/s
Downloading: ncbi_dataset.zip    29MB 17.5MB/s
Downloading: ncbi_dataset.zip    29.2MB 17.5MB/s
Downloading: ncbi_dataset.zip    29.4MB 17.5MB/s
Downloading: ncbi_dataset.zip    29.5MB 17.6MB/s
Downloading: ncbi_dataset.zip    29.8MB 17.6MB/s
Downloading: ncbi_dataset.zip    30MB 17.6MB/s
Downloading: ncbi_dataset.zip    30.2MB 17.6MB/s
Downloading: ncbi_dataset.zip    30.3MB 17.6MB/s
Downloading: ncbi_dataset.zip    30.6MB done
New version of client (12.7.0)

In [9]:
!unzip -o ncbi_dataset.zip -d PtRefSeq

Archive:  ncbi_dataset.zip
  inflating: PtRefSeq/README.md      
  inflating: PtRefSeq/ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: PtRefSeq/ncbi_dataset/data/GCF_000002775.4/chr5.fna  
  inflating: PtRefSeq/ncbi_dataset/data/GCF_000002775.4/genomic.gff  
  inflating: PtRefSeq/ncbi_dataset/data/GCF_000002775.4/genomic.gtf  
  inflating: PtRefSeq/ncbi_dataset/data/GCF_000002775.4/sequence_report.jsonl  
  inflating: PtRefSeq/ncbi_dataset/data/dataset_catalog.json  


Magic-BLAST requires a BLAST database in order to align the reads.

In [10]:
!makeblastdb -in PtRefSeq/ncbi_dataset/data/GCF_000002775.4/chr5.fna -dbtype nucl -parse_seqids -out PtRefSeq/Pt.chrom5




Building a new DB, current time: 08/16/2021 18:45:42
New DB name:   /home/jovyan/PtRefSeq/Pt.chrom5
New DB title:  PtRefSeq/ncbi_dataset/data/GCF_000002775.4/chr5.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.510293 seconds.




## Align Fastq Files to Reference Genome to Produce Bams using Magic-BLAST

Magic-BLAST is a tool for mapping large next-generation RNA or DNA sequencing runs against a whole genome or transcriptome. Each alignment optimizes a composite score, taking into account simultaneously the two reads of a pair, and in case of RNA-seq, locating the candidate introns and adding up the score of all exons. This is very different from other versions of BLAST, where each exon is scored as a separate hit and read-pairing is ignored.

Create a location to store the bams.

In [11]:
!mkdir bams

This next command limits the number of bases that are input into the magicblast command before running it. The default is 50M. The only reason the value is being set lower is to allow this notebook to work on the binder server, which as a 2 GB virtual memory limit. If the limit is still too high when running the notebook, try reducing the BATCH_SIZE to 5000000. 

In [12]:
%env BATCH_SIZE=7500000

env: BATCH_SIZE=7500000


Magic-BLAST has a variety of parameters that can be set. In this example, we are setting the parameters:

* *splice* is *True* (transcriptomic data)
* *no_unaligned* (to reduce the file size that is generated)
* *max_db_word_count* is 10 to reduce virtual memory demand (Magicblast rejects 16-base word matches between input reads and the target database if this word appears in the database more than <number> of times. The default is 30.

Samtools

The default output type for MagicBlast is a sam file, so we pipe it into samtools to make a bam for later viewing on the Genome Data Viewer. The sort command has a large virtual memory command.

* *-m* parameter is set to 250M to reduce load in this binder notebook (it can be removed on your own machine)

In [13]:
!magicblast -query data/SRR3241538.fastq -db PtRefSeq/Pt.chrom5 -splice T -no_unaligned -infmt fastq -max_db_word_count 10 | \
samtools view -bS | samtools sort -m 250M -o bams/SRR3241538.bam -O BAM

The bam files need to be indexed to be used in Genome Data Viewer.

In [15]:
!for i in bams/*.bam; do samtools index $i; done

In [1]:
!ls bams

SRR3241538.bam	SRR3241538.bam.bai


In [14]:
#If you are using your own accession list, you can generate the list of command by running this bash command after removing the "#" from the beginning of each line

#!cat accessionlist.txt| while read line;do \
#echo "magicblast -query data/$line.fastq -db Pt.chrom5 -splice T -no_unaligned -infmt fastq -max_db_word_count 10 | \
#samtools view -bS | samtools sort -m 250M -o bams/$line.bam -O BAM"; done

For this demo, we have already generated the bams and indexed bam files. They can be copied into your bam directory by running the following command.

In [25]:
!ls bams

SRR3241535.bam	SRR3241538.bam	SRR3241538.bam.bai


In [27]:
!cat accessionlist.txt | xargs -I % wget -O bams/%.bam https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/%.bam
!cat accessionlist.txt | xargs -I % wget -O bams/%.bam.bai https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/%.bam.bai

--2021-08-16 20:56:08--  https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241527.bam
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 130.14.250.7, 2607:f220:41e:250::12, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5216697 (5.0M) [application/x-bam]
Saving to: ‘bams/SRR3241527.bam’

bams/SRR3241527.bam 100%[===================>]   4.97M  15.0MB/s    in 0.3s    

2021-08-16 20:56:09 (15.0 MB/s) - ‘bams/SRR3241527.bam’ saved [5216697/5216697]

--2021-08-16 20:56:09--  https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241528.bam
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 165.112.9.229, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Le

HTTP request sent, awaiting response... 200 OK
Length: 25168 (25K) [application/x-bam]
Saving to: ‘bams/SRR3241528.bam.bai’

bams/SRR3241528.bam 100%[===================>]  24.58K  --.-KB/s    in 0.03s   

2021-08-16 20:56:14 (918 KB/s) - ‘bams/SRR3241528.bam.bai’ saved [25168/25168]

--2021-08-16 20:56:14--  https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241529.bam.bai
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 130.14.250.7, 2607:f220:41e:250::12, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25408 (25K) [application/x-bam]
Saving to: ‘bams/SRR3241529.bam.bai’

bams/SRR3241529.bam 100%[===================>]  24.81K  --.-KB/s    in 0.02s   

2021-08-16 20:56:14 (1003 KB/s) - ‘bams/SRR3241529.bam.bai’ saved [25408/25408]

--2021-08-16 20:56:14--  https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/0

In [28]:
!ls bams

SRR3241527.bam	    SRR3241530.bam	SRR3241533.bam	    SRR3241536.bam
SRR3241527.bam.bai  SRR3241530.bam.bai	SRR3241533.bam.bai  SRR3241536.bam.bai
SRR3241528.bam	    SRR3241531.bam	SRR3241534.bam	    SRR3241537.bam
SRR3241528.bam.bai  SRR3241531.bam.bai	SRR3241534.bam.bai  SRR3241537.bam.bai
SRR3241529.bam	    SRR3241532.bam	SRR3241535.bam	    SRR3241538.bam
SRR3241529.bam.bai  SRR3241532.bam.bai	SRR3241535.bam.bai  SRR3241538.bam.bai


To be continued ....